In [1]:
#%pip install python-decouple

In [ ]:
#import libraries
from decouple import config
from ibm_watsonx_ai import APIClient
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.foundation_models.prompts import PromptTemplate, PromptTemplateManager
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes
from ibm_watsonx_ai.foundation_models.schema import TextGenParameters
import pandas as pd
from sklearn.metrics import classification_report 
import import_ipynb
import Train_Test_split as ts

In [2]:
#get the API key and project key
WX_API_KEY = config('WX_API_KEY')
PROJECT_ID = "f79c2d38-7ee2-4de6-931a-dad71b72d34f"

In [3]:
credentials = Credentials(
    url = "https://us-south.ml.cloud.ibm.com",
    api_key = WX_API_KEY
)

client = APIClient(
    credentials=credentials, 
    project_id=PROJECT_ID
)

In [ ]:
PARAMS = TextGenParameters(
    temperature=0.2,      # Higher temperature means more randomness
    max_new_tokens=3, # Maximum number of tokens to generate
    min_new_tokens=1, # Minimum number of tokens to generate
)

model = ModelInference(
    api_client=client,
    model_id="mistralai/mistral-small-3-1-24b-instruct-2503",
    params=PARAMS
)
TextGenParameters.show()

In [ ]:
#This is partly generated using Copilot suggestions

def sentiment_prompt(text):
    prompt = """ 
    Return only a number based on the sentiment for the text. Do not give an explanation. Your response should only be a single character:
    1 for Negative
    2 for Neural
    3 for Positive.

    text: {}

    """
    return prompt.format(text).replace('\n', '')

In [ ]:
#This is also generated using Copilot suggestions

def get_sentiment_llm(review):
    prompt = sentiment_prompt(review)
    response = model.generate_text(prompt)
    return response

In [ ]:
df = ts.BI_df[["business_id", "text"]]
df

,business_id,text
31,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST..."
46,MjZQqZAmJeMco_Vq-Y9h-g,Drum-roll please! Review #100 coming right up!...
54,u7MJKcNdZXYyTeb67vD5jw,We stopped here for my Chai and Hubby's coffee...
89,saJFbz12EnzanelpD8_xXQ,There's been three times that I've ordered a g...
104,KiE0h68HGOO7ZXAqkMBdiw,"I went in when they had 4 people working, wait..."
...,...,...
586567,-85kJMtb9wqNWDT8yLbitw,Fair service- messed up my drink cuppa times....
586568,Ddg-J_j0YFErk7wpMtH_0A,On my way out of Reno last week I noted a new ...
586638,2rmpfdyV2POqpXtmPHO_IQ,always closing drive thru. i've driven here so...
586646,GxuxCctcz3Hyk0wnuly7vQ,This is now one of two Starbucks in the shoppi...


In [ ]:
df1 = df.iloc[0:5000].copy()
df1["llm_sentiment"] = df1["text"].apply(get_sentiment_llm)

In [ ]:
df2 = df.iloc[5000:10000].copy()
df2["llm_sentiment"] = df2["text"].apply(get_sentiment_llm)

,business_id,text,llm_sentiment
31,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",1


In [ ]:
df3 = df.iloc[10000:15000].copy()
df3["llm_sentiment"] = df3["text"].apply(get_sentiment_llm)

,business_id,text,llm_sentiment
31,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",1


In [ ]:
df4 = df.iloc[15000:].copy()
df4["llm_sentiment"] = df4["text"].apply(get_sentiment_llm)

,business_id,text,llm_sentiment
31,aJvxWyQIG5OLfBw3qAe8xA,"Ordered Caramel frappe at Drive thru, BIG MIST...",1


In [ ]:
final_df = pd.concat([df1,df2,df3,df4])

In [ ]:
final_df.info()

In [ ]:
final_df.head(10)